In [1]:
import os
import random
from moviepy.editor import *
import moviepy.audio.fx.all as afx
from moviepy.video import fx
from moviepy.config import change_settings
change_settings({"FFMPEG_BINARY":"ffmpeg"})
# from PIL import Image

In [2]:
select_folder = int(input('Select folder: '))

In [3]:
if select_folder == 1:
    folder_test = 'History'
elif select_folder == 2:
    folder_test = 'Music'
elif select_folder == 3:
    folder_test = 'Tech'
else:
    pass

In [4]:
carpeta = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/4_create_video_by_news/videos_by_news'
intro_path = f'/mnt/D8E84E4DE84E2A58/backUp_Img_Vid/Intro/{folder_test}/Intro.mp4'
video_transicion_path = "transicion.png"

In [5]:
# def combinar_videos_con_fade(carpeta, num_archivos=20, duracion_fade=1):
#     # Seleccionar archivos de video en orden
#     archivos = sorted([f for f in os.listdir(carpeta) if f.endswith('.mp4')])
#     if len(archivos) == 0:
#         raise ValueError(f'La carpeta {carpeta} está vacía o no contiene archivos válidos.')

#     # Limitar la cantidad de archivos
#     archivos = archivos[:num_archivos]
#     ruta_archivos = [os.path.join(carpeta, archivo) for archivo in archivos]

#     # Cargar clips y añadir fades
#     clips = []
#     for ruta in ruta_archivos:
#         clip = VideoFileClip(ruta)
#         clip = clip.fadein(duracion_fade).fadeout(duracion_fade)  # Transiciones
#         clips.append(clip)

#     # Concatenar videos
#     video_final = concatenate_videoclips(clips, method="compose")
#     return video_final


# def combinar_videos_con_fade(carpeta, num_archivos=20, duracion_fade=1, duracion_transicion=1):
#     # Seleccionar archivos de video en orden
#     archivos = sorted([f for f in os.listdir(carpeta) if f.endswith('.mp4')])
#     if len(archivos) == 0:
#         raise ValueError(f'La carpeta {carpeta} está vacía o no contiene archivos válidos.')

#     # Limitar la cantidad de archivos
#     archivos = archivos[:num_archivos]
#     ruta_archivos = [os.path.join(carpeta, archivo) for archivo in archivos]

#     # Crear una transición personalizada (ColorClip como ejemplo)
#     transicion = ColorClip(size=(1920, 1080), color=(0, 0, 0), duration=duracion_transicion).fadein(duracion_fade).fadeout(duracion_fade)

#     # Cargar clips y añadir la transición personalizada entre ellos
#     clips_con_transiciones = []
#     for i, ruta in enumerate(ruta_archivos):
#         clip = VideoFileClip(ruta).fadein(duracion_fade).fadeout(duracion_fade)
#         clips_con_transiciones.append(clip)
#         if i < len(ruta_archivos) - 1:  # Añadir transición después de cada clip excepto el último
#             clips_con_transiciones.append(transicion)

#     # Concatenar todos los clips (videos + transiciones)
#     video_final = concatenate_videoclips(clips_con_transiciones, method="compose")
#     return video_final



def combinar_videos_con_fade(carpeta, num_archivos=20, duracion_fade=1, duracion_transicion=1, transicion_path="transicion.png"):
    # Seleccionar archivos de video en orden
    archivos = sorted([f for f in os.listdir(carpeta) if f.endswith('.mp4')])
    if len(archivos) == 0:
        raise ValueError(f'La carpeta {carpeta} está vacía o no contiene archivos válidos.')

    # Limitar la cantidad de archivos
    archivos = archivos[:num_archivos]
    ruta_archivos = [os.path.join(carpeta, archivo) for archivo in archivos]

    # Cargar la imagen de transición generada y crear un clip de video a partir de ella
    if not os.path.exists(transicion_path):
        raise FileNotFoundError(f'No se encontró la imagen de transición en {transicion_path}')
    
    transicion_imagen = ImageClip(transicion_path, duration=duracion_transicion).fadein(duracion_fade).fadeout(duracion_fade)

    # Cargar clips y añadir la transición personalizada entre ellos
    clips_con_transiciones = []
    for i, ruta in enumerate(ruta_archivos):
        clip = VideoFileClip(ruta).fadein(duracion_fade).fadeout(duracion_fade)
        clips_con_transiciones.append(clip)
        if i < len(ruta_archivos) - 1:  # Añadir transición después de cada clip excepto el último
            clips_con_transiciones.append(transicion_imagen)

    # Concatenar todos los clips (videos + transiciones)
    video_final = concatenate_videoclips(clips_con_transiciones, method="compose")
    return video_final

In [6]:
# try:
#     # Combinar videos con transiciones
#     video_final = combinar_videos_con_fade(carpeta, num_archivos=10, duracion_fade=1.5)

#     # Cargar intro y combinar con el video final
#     intro = VideoFileClip(intro_path).set_duration(5)  # Ajustar duración del intro si es necesario
#     final_video = concatenate_videoclips([intro, video_final], method="compose")

#     # Guardar el video final
#     final_video.write_videofile('video_automatico.mp4', codec="libx264", audio_codec="aac", fps=24)
#     print("¡Listo! Video automático creado con éxito.")
# except ValueError as e:
#     print(f"Error: {e}")
# except FileNotFoundError as e:
#     print(f"No se encontró el archivo: {e}")
# except Exception as e:
#     print(f"Error inesperado: {e}")


# Bloque de uso
try:
    # Combinar videos con transiciones
    video_final = combinar_videos_con_fade(carpeta, num_archivos=10, duracion_fade=1.5)

    # Cargar intro y combinar con el video final
    intro = VideoFileClip(intro_path).set_duration(5)  # Ajustar duración del intro si es necesario
    final_video = concatenate_videoclips([intro, video_final], method="compose")

    # Guardar el video final
    # final_video.write_videofile('video_automatico.mp4', codec="libx264", audio_codec="aac", fps=24)
    final_video.write_videofile('video_automatico.mp4', codec="h264_nvenc", audio_codec="aac", fps=24)
    print("¡Listo! Video automático creado con éxito.")
except ValueError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"No se encontró el archivo: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")

Moviepy - Building video video_automatico.mp4.
MoviePy - Writing audio in video_automaticoTEMP_MPY_wvf_snd.mp4


chunk:  38%|███▊      | 8098/21137 [00:05<00:09, 1340.44it/s, now=None]

KeyboardInterrupt: 

chunk:  39%|███▉      | 8217/21137 [00:19<00:09, 1340.44it/s, now=None]